# Model

The goal is to understand the model and its manipulations.
From YOLOV8, the project source code were moved to ultralytis and rearanged compared to YOLOV5.

Here, we explore how we can manipulate the models to fit the usages.

## 1. Model files

In [19]:
# if not in the yolov10 project folder, should set the python path
# This is not needed if using pypi's version

import sys
sys.path.append("/tmp/yolov10/yolov10")

# set root folder for asset
root = "/tmp/yolov10"

In [1]:
# Load a model without pretrained weights

# The .yaml files contain the definitions of the yolo models. 
# When loading model, the program will parse these yaml files and generate pytorch models.
# Those predefined yaml files are in: ultralytics/cfg/models/v10/ 

# The model in this case has random weights and can not be used.

from ultralytics import YOLOv10 
model = YOLOv10("yolov10s.yaml")
model

[W904 09:23:12.828466237 NNPACK.cpp:61] Could not initialize NNPACK! Reason: Unsupported hardware.


YOLOv10(
  (model): YOLOv10DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track

In [ ]:
# To load a pretrained model in the Huggingface repo

model = YOLOv10.from_pretrained(ckp)

In [ ]:
# to load a local pretrained model
# to download the models: https://github.com/THU-MIG/yolov10/releases/.

model_local = YOLOv10("local_path_to_model")

In [ ]:
# we can also load a model 
model = YOLOv10("yolov10s.yaml")

# then load the weights
model.load("yolov10s.pt")

Some useful locations:
 - engine/model.py: contains the Model class with all its functions such as predict, train...
 - nn/tasks.py: utility functions such as parse_model to parse yaml model file to generate pytorch model
 - nn/modules/blacks.py: contains class definitions of submodel used in yaml model definitions.

To use custom submodels, one can:
 - give the definition of the block in nn/modules/block.py
 - add the definition in the submodel list in nn/tasks.py
 - insert as a layer in the yaml file

# 2. Introduce a new layer

1. get a block class definition from: https://github.com/ZhugeKongan/Attention-mechanism-implementation/blob/main/models/SE_block.py
2. add this block class to nn/modules/block.py. 
3. Different from YOLOV5 which import everything, YOLOv10 import each modules, so we have to add the new definition to the import list (in this case SE):
    - nn/modules/__init__.py
    - nn/tasks.py

4. In nn/tasks.py 'parse_model' function, add the new definition and specify its way of parsing:
    ```python
    elif m is SE:
        c1 = ch[f]
        c2 = args[0]
        ...
    ```
5. Add layer in the .yaml file for model definitions: ultralytics/cfg/models/v10/*.yaml
6. Load the model with new layer definition:
    ```python
    model = YOLOv10("path_to_model_yaml/your_model.yaml")
    ```
7. Load weights, this is optional, but will simplify the training:
    ```python
    model.load("yolov10s.pt")
    ```
8. train the model as pretrained ones. For training detail, see 'yolo-v10_train.ipynb', section 'Custom Layer'.

In [1]:
# load the model and transfer the available weights
# In the model definition, we can see that the 11th layer is SE now.

from ultralytics import YOLOv10 
model = YOLOv10("yolov10s_se.yaml")
model.load("yolov10s.pt")
model

[W909 11:18:37.823954649 NNPACK.cpp:61] Could not initialize NNPACK! Reason: Unsupported hardware.
/home/niuniu/Documents/DL_pytorch/yolov10/yolov10/ultralytics/nn/tasks.py:734: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for 

Transferred 250/623 items from pretrained weights


YOLOv10(
  (model): YOLOv10DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(96, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track

# 3. Change backbone

To change the backbone of the model follow almost the same steps as add new layers (eg. layer definition, define args and change yaml file.)
As the input size of the model is 640X640, so the changed backbone should output the sizes 80X80, 40X40 and 20X20 which will be concatenated to the heads.